## Ch3 MNIST CNN

### 1. Low level API 사용

In [1]:
import tensorflow as tf

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

W0809 15:38:56.366904 140665456572224 deprecation.py:323] From <ipython-input-1-bed9dddec7b6>:4: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0809 15:38:56.367622 140665456572224 deprecation.py:323] From /home/john/anaconda3/envs/tf_server/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0809 15:38:56.367963 140665456572224 deprecation.py:323] From /home/john/anaconda3/envs/tf_server/lib/python3.7/site-packages/tensorflow/contrib/learn/python/learn/datasets/mnist.py:262: extract_images (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated an

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [2]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

In [3]:
W1 = tf.Variable(tf.random_normal([3,3,1,32], stddev=0.01))
L1 = tf.nn.conv2d(X, W1, strides=[1,1,1,1], padding='SAME')
L1 = tf.nn.relu(L1)
L1 = tf.nn.max_pool(L1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W2 = tf.Variable(tf.random_normal([3,3,32,64], stddev=0.01))
L2 = tf.nn.conv2d(L1, W2, strides=[1,1,1,1], padding='SAME')
L2 = tf.nn.relu(L2)
L2 = tf.nn.max_pool(L2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

W3 = tf.Variable(tf.random_normal([7*7*64, 256], stddev=0.01))
L3 = tf.reshape(L2, [-1, 7*7*64])
L3 = tf.matmul(L3, W3)
L3 = tf.nn.relu(L3)
L3 = tf.nn.dropout(L3, keep_prob)

W4 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
model = tf.matmul(L3, W4)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

W0809 15:35:56.134239 140511702153024 deprecation.py:506] From <ipython-input-3-a27294ff6a9b>:15: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [4]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

In [5]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [6]:
for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, 
                                                             Y: batch_ys,
                                                             keep_prob: 0.7})
        
        total_cost += cost_val
    print('Epoch: %04d' %(epoch+1),
          'Avg. cost: %.3f' % float(total_cost/total_batch))
    
print("Done")

Epoch: 0001 Avg. cost: 0.352
Epoch: 0002 Avg. cost: 0.110
Epoch: 0003 Avg. cost: 0.080
Epoch: 0004 Avg. cost: 0.064
Epoch: 0005 Avg. cost: 0.052
Epoch: 0006 Avg. cost: 0.042
Epoch: 0007 Avg. cost: 0.037
Epoch: 0008 Avg. cost: 0.032
Epoch: 0009 Avg. cost: 0.028
Epoch: 0010 Avg. cost: 0.027
Done


In [8]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: ', sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1), Y: mnist.test.labels, keep_prob: 1.}))

sess.close()

정확도:  0.9901


### 2. High level API 사용
- tf.keras.layer 사용해서 추가 필요

In [3]:
X = tf.placeholder(tf.float32, [None, 28, 28, 1])
Y = tf.placeholder(tf.float32, [None, 10])
is_training = tf.placeholder(tf.bool)

In [7]:
L1 = tf.layers.conv2d(X, 32, [3,3])
L1 = tf.layers.max_pooling2d(L1, [2,2], [2,2])
L1 = tf.layers.dropout(L1, 0.7, is_training)

L2 = tf.layers.conv2d(L1, 64, [3,3])
L2 = tf.layers.max_pooling2d(L2, [2,2], [2,2])
L2 = tf.layers.dropout(L2, 0.7, is_training)

L3 = tf.contrib.layers.flatten(L2)
L3 = tf.layers.dense(L3, 256, activation=tf.nn.relu)
L3 = tf.layers.dropout(L3, 0.5, is_training)

model = tf.layers.dense(L3, 10, activation=None)

W0809 15:43:07.808753 140665456572224 deprecation.py:323] From /home/john/anaconda3/envs/tf_server/lib/python3.7/site-packages/tensorflow/contrib/layers/python/layers/layers.py:1634: flatten (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.flatten instead.
W0809 15:43:07.914605 140665456572224 deprecation.py:323] From <ipython-input-7-098ab132edfe>:10: dense (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.dense instead.


In [8]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

In [9]:
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.Session(config=config)
sess.run(tf.global_variables_initializer())

In [10]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

In [13]:
for epoch in range(10):
    total_cost = 0
    
    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        batch_xs = batch_xs.reshape(-1, 28, 28, 1)
        
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, 
                                                             Y: batch_ys,
                                                             is_training:True})
        
        total_cost += cost_val
    print('Epoch: %04d' %(epoch+1),
          'Avg. cost: %.3f' % float(total_cost/total_batch))
    
print("Done")

Epoch: 0001 Avg. cost: 0.006
Epoch: 0002 Avg. cost: 0.005
Epoch: 0003 Avg. cost: 0.009
Epoch: 0004 Avg. cost: 0.004
Epoch: 0005 Avg. cost: 0.004
Epoch: 0006 Avg. cost: 0.008
Epoch: 0007 Avg. cost: 0.005
Epoch: 0008 Avg. cost: 0.003
Epoch: 0009 Avg. cost: 0.005
Epoch: 0010 Avg. cost: 0.005
Done


In [14]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: ', sess.run(accuracy, feed_dict={X: mnist.test.images.reshape(-1, 28, 28, 1), 
                                              Y: mnist.test.labels, 
                                              is_training: False}))

sess.close()

정확도:  0.9893
